In [177]:
import pandas as pd
import numpy as np

# Excel to Python Lesson 2: Pandas in the Mist

Lesson 1 focused on getting your data loaded into Pandas. 
Now, we'll start manipulating that data into something useful.

This lesson is meant to be introductory level and will act as a survey of things that are possible.
Future lessons will get into the digital weeds.

Lesson Topics:
* Simple Data Cleansing
    * Handling Nulls
    * Handling Duplicates
    * Handling Duplicates
* Table Manipulations
    * Splitting Fields
    * Dropping Columns
    * Aggregations
* Combining Tables
    * Concat/Append - Combining Data
    * Merge - Database Style Joins
* Excel Lyfe
    * VLOOKUP
    * Pivot Tables
    
    
    

In [126]:
# load data to be used

simple1_df = pd.read_csv('simple_data_cleansing_1.csv')
simple2_df = pd.read_csv('simple_data_cleansing_2.csv')

solo_df = pd.read_csv('solo_table.tsv', sep='\t')

concat_a_df = pd.read_csv('concat_table_a.tsv', sep='\t')
concat_b_df = pd.read_csv('concat_table_b.tsv', sep='\t')

concat_c_df = pd.read_csv('concat_table_c.tsv', sep='\t')


## Simple Data Cleansing

### Handling Nulls
`df.dropna() # drops if anything in the row is null`

`df.dropna(how='all') # drops if the entire row is null`

`df.dropna(axis=1, how='all') # drops columns where all data is null`

`df.fillna(0, inplace=True)`
`df.fillna({1: 'X', 2: 0}, inplace=True)`



In [19]:
# Let's look at the data we have
simple1_df

,name,favorite_bird,allergic_to_unicorns_flag
0,Santos Halper,raven,Y
1,Robert McFly,eagle,N
2,Robert McFly,eagle,N
3,Person McPersonface,NaN,Y
4,Lando Kal,finch,Y
5,NaN,NaN,NaN
6,NaN,raven,Y
7,Sar Dean,pelican,Y
8,Sar Dean,dove,Y


In [20]:
# This will drop any rows that have any nulls
simple1_df.dropna()

,name,favorite_bird,allergic_to_unicorns_flag
0,Santos Halper,raven,Y
1,Robert McFly,eagle,N
2,Robert McFly,eagle,N
4,Lando Kal,finch,Y
7,Sar Dean,pelican,Y
8,Sar Dean,dove,Y


In [21]:
# Let's remove only the row that had all nulls
simple1_df.dropna(how='all')

,name,favorite_bird,allergic_to_unicorns_flag
0,Santos Halper,raven,Y
1,Robert McFly,eagle,N
2,Robert McFly,eagle,N
3,Person McPersonface,NaN,Y
4,Lando Kal,finch,Y
6,NaN,raven,Y
7,Sar Dean,pelican,Y
8,Sar Dean,dove,Y


In [28]:
# what if a column was all nulls?
simple2_df

,name,favorite_bird,allergic_to_unicorns_flag,favorite_color
0,Santos Halper,raven,Y,NaN
1,Robert McFly,eagle,N,NaN
2,Robert McFly,eagle,N,NaN
3,Person McPersonface,NaN,Y,NaN
4,Lando Kal,finch,Y,NaN
5,NaN,NaN,NaN,NaN
6,NaN,raven,Y,NaN
7,Sar Dean,pelican,Y,NaN
8,Sar Dean,dove,Y,NaN


In [33]:
# Let's remove favorite_color as it's all nulls
new_simple_df = simple2_df.dropna(axis=1, how='all')
new_simple_df

,name,favorite_bird,allergic_to_unicorns_flag
0,Santos Halper,raven,Y
1,Robert McFly,eagle,N
2,Robert McFly,eagle,N
3,Person McPersonface,NaN,Y
4,Lando Kal,finch,Y
5,NaN,NaN,NaN
6,NaN,raven,Y
7,Sar Dean,pelican,Y
8,Sar Dean,dove,Y


In [37]:
# let's drop that null row
new_simple_df = new_simple_df.dropna(how='all')
new_simple_df

,name,favorite_bird,allergic_to_unicorns_flag
0,Santos Halper,raven,Y
1,Robert McFly,eagle,N
2,Robert McFly,eagle,N
3,Person McPersonface,NaN,Y
4,Lando Kal,finch,Y
6,NaN,raven,Y
7,Sar Dean,pelican,Y
8,Sar Dean,dove,Y


In [38]:
# Let's replace the null bird values with wren
new_simple_df.fillna('wren')

,name,favorite_bird,allergic_to_unicorns_flag
0,Santos Halper,raven,Y
1,Robert McFly,eagle,N
2,Robert McFly,eagle,N
3,Person McPersonface,wren,Y
4,Lando Kal,finch,Y
6,wren,raven,Y
7,Sar Dean,pelican,Y
8,Sar Dean,dove,Y


In [45]:
# Let's try again by specifying the column to replace
new_simple_df.fillna({'favorite_bird':'wren'}, inplace=True)
new_simple_df

In [46]:
new_simple_df

,name,favorite_bird,allergic_to_unicorns_flag
0,Santos Halper,raven,Y
1,Robert McFly,eagle,N
2,Robert McFly,eagle,N
3,Person McPersonface,wren,Y
4,Lando Kal,finch,Y
6,NaN,raven,Y
7,Sar Dean,pelican,Y
8,Sar Dean,dove,Y


In [47]:
# And do the same for the name
new_simple_df.fillna({'name':'Anonymous User'}, inplace=True)
new_simple_df

,name,favorite_bird,allergic_to_unicorns_flag
0,Santos Halper,raven,Y
1,Robert McFly,eagle,N
2,Robert McFly,eagle,N
3,Person McPersonface,wren,Y
4,Lando Kal,finch,Y
6,Anonymous User,raven,Y
7,Sar Dean,pelican,Y
8,Sar Dean,dove,Y


### Handling Duplicates

`df.drop_duplicates()  # drops the pure duplicates`

`df.drop_duplicates(['col1', 'col2'], keep='last')`

In [48]:
new_simple_df.drop_duplicates()

,name,favorite_bird,allergic_to_unicorns_flag
0,Santos Halper,raven,Y
1,Robert McFly,eagle,N
3,Person McPersonface,wren,Y
4,Lando Kal,finch,Y
6,Anonymous User,raven,Y
7,Sar Dean,pelican,Y
8,Sar Dean,dove,Y


In [51]:
new_simple_df.drop_duplicates(['name'], keep='last')

,name,favorite_bird,allergic_to_unicorns_flag
0,Santos Halper,raven,Y
2,Robert McFly,eagle,N
3,Person McPersonface,wren,Y
4,Lando Kal,finch,Y
6,Anonymous User,raven,Y
8,Sar Dean,dove,Y


## Single Table Manipulations


In [95]:
solo_df.head()

,transaction_id,quarter,customer_name,transit_code,order_total_amt
0,3345,Q1,Robert Robertson,X4,100
1,3346,Q1,David Davison,Y4,230
2,3347,Q1,Jack Froot,X5,332
3,3348,Q1,Dolby Sorand,Y5,50
4,3349,Q1,Robert Robertson,X6,65


#### Splitting Columns

In [101]:
# Let's start by splitting the customer_name field into a first_name and last_name
solo_df[['first_name', 'last_name']] = solo_df['customer_name'].str.split(' ', expand=True)
solo_df.head()

,transaction_id,quarter,customer_name,transit_code,order_total_amt,first_name,last_name
0,3345,Q1,Robert Robertson,X4,100,Robert,Robertson
1,3346,Q1,David Davison,Y4,230,David,Davison
2,3347,Q1,Jack Froot,X5,332,Jack,Froot
3,3348,Q1,Dolby Sorand,Y5,50,Dolby,Sorand
4,3349,Q1,Robert Robertson,X6,65,Robert,Robertson


#### Dropping Columns

In [102]:
# Let's get rid of customer_name and transit_code
solo_df.drop(['customer_name', 'transit_code'], axis=1, inplace=True)
solo_df.head()

,transaction_id,quarter,order_total_amt,first_name,last_name
0,3345,Q1,100,Robert,Robertson
1,3346,Q1,230,David,Davison
2,3347,Q1,332,Jack,Froot
3,3348,Q1,50,Dolby,Sorand
4,3349,Q1,65,Robert,Robertson


#### Aggregations

##### Order Total Amount by Quarter

If this was SQL, we could do like this:

```
SELECT quarter, SUM(order_total_amt)
FROM solo_df
GROUP BY quarter;
```


In [104]:
solo_df.groupby('quarter').sum()

,transaction_id,order_total_amt
quarter,,
Q1,70455,4552
Q2,57358,4772
Q3,30483,2125
Q4,27164,1525


In [105]:
solo_df.groupby('quarter').order_total_amt.sum()

quarter
Q1    4552
Q2    4772
Q3    2125
Q4    1525
Name: order_total_amt, dtype: int64

In [106]:
solo_df.groupby('quarter').order_total_amt.agg(min_order='min', max_order='max', avg_order='mean', total_order='sum')

,min_order,max_order,avg_order,total_order
quarter,,,,
Q1,50,567,216.761905,4552
Q2,50,567,280.705882,4772
Q3,50,567,236.111111,2125
Q4,50,543,190.625000,1525


In [111]:
solo_df.groupby(['last_name']).order_total_amt.sum()

last_name
Davison      3818
Froot        3621
Robertson    2213
Sorand       3322
Name: order_total_amt, dtype: int64

In [117]:
solo_df['running_total'] = solo_df.order_total_amt.cumsum()
solo_df.head(20)

,transaction_id,quarter,order_total_amt,first_name,last_name,running_total
0,3345,Q1,100,Robert,Robertson,100
1,3346,Q1,230,David,Davison,330
2,3347,Q1,332,Jack,Froot,662
3,3348,Q1,50,Dolby,Sorand,712
4,3349,Q1,65,Robert,Robertson,777
5,3350,Q1,80,David,Davison,857
6,3351,Q1,125,Jack,Froot,982
7,3352,Q1,225,Dolby,Sorand,1207
8,3353,Q1,347,Robert,Robertson,1554
9,3354,Q1,567,David,Davison,2121


## Combining Tables

### Concat - Combining Data

#### Add more of the same data to the end of a dataframe

In [120]:
concat_a_df.head()

,transaction_id,quarter,customer_name,transit_code,order_total_amt
0,3345,Q1,Robert Robertson,X4,100
1,3346,Q1,David Davison,Y4,230
2,3347,Q1,Jack Froot,X5,332
3,3348,Q1,Dolby Sorand,Y5,50
4,3349,Q1,Robert Robertson,X6,65


In [121]:
concat_b_df.head()

,transaction_id,quarter,customer_name,transit_code,order_total_amt
0,3366,Q2,David Davison,Y14,567
1,3367,Q2,Jack Froot,X15,334
2,3368,Q2,Dolby Sorand,Y15,543
3,3369,Q2,Robert Robertson,X16,100
4,3370,Q2,David Davison,Y16,230


In [124]:
combined = pd.concat([concat_a_df, concat_b_df], ignore_index=True)
combined

,transaction_id,quarter,customer_name,transit_code,order_total_amt
0,3345,Q1,Robert Robertson,X4,100
1,3346,Q1,David Davison,Y4,230
2,3347,Q1,Jack Froot,X5,332
3,3348,Q1,Dolby Sorand,Y5,50
4,3349,Q1,Robert Robertson,X6,65
5,3350,Q1,David Davison,Y6,80
6,3351,Q1,Jack Froot,X7,125
7,3352,Q1,Dolby Sorand,Y7,225
8,3353,Q1,Robert Robertson,X8,347
9,3354,Q1,David Davison,Y8,567


In [127]:
concat_c_df

,process_flag
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,2
8,2
9,0


In [128]:
# What happens here?
pd.concat([concat_a_df, concat_c_df])

,transaction_id,quarter,customer_name,transit_code,order_total_amt,process_flag
0,3345.0,Q1,Robert Robertson,X4,100.0,NaN
1,3346.0,Q1,David Davison,Y4,230.0,NaN
2,3347.0,Q1,Jack Froot,X5,332.0,NaN
3,3348.0,Q1,Dolby Sorand,Y5,50.0,NaN
4,3349.0,Q1,Robert Robertson,X6,65.0,NaN
5,3350.0,Q1,David Davison,Y6,80.0,NaN
6,3351.0,Q1,Jack Froot,X7,125.0,NaN
7,3352.0,Q1,Dolby Sorand,Y7,225.0,NaN
8,3353.0,Q1,Robert Robertson,X8,347.0,NaN
9,3354.0,Q1,David Davison,Y8,567.0,NaN


In [129]:
# How about now?
pd.concat([concat_a_df, concat_c_df], axis=1)

,transaction_id,quarter,customer_name,transit_code,order_total_amt,process_flag
0,3345,Q1,Robert Robertson,X4,100,1
1,3346,Q1,David Davison,Y4,230,1
2,3347,Q1,Jack Froot,X5,332,1
3,3348,Q1,Dolby Sorand,Y5,50,1
4,3349,Q1,Robert Robertson,X6,65,1
5,3350,Q1,David Davison,Y6,80,1
6,3351,Q1,Jack Froot,X7,125,1
7,3352,Q1,Dolby Sorand,Y7,225,2
8,3353,Q1,Robert Robertson,X8,347,2
9,3354,Q1,David Davison,Y8,567,0


### Merge - Database Style Joins

In [148]:
tx_df = pd.read_csv('tx_table.tsv', sep='\t')
shipment_df = pd.read_csv('shipment_table.tsv', sep='\t')

In [149]:
tx_df.head()

,transaction_id,quarter,customer_name,transit_code,order_total_amt,process_flag
0,3345,Q1,Robert Robertson,X4,100,1
1,3346,Q1,David Davison,Y4,230,1
2,3347,Q1,Jack Froot,X5,332,1
3,3348,Q1,Dolby Sorand,Y5,50,1
4,3349,Q1,Robert Robertson,X6,65,1


In [150]:
shipment_df.head()

,shipment_id,transaction_id,shipment_status
0,xk-9923,3345,in_process
1,xk-9924,3346,shipped
2,xk-9925,3347,shipped
3,xk-9926,3348,delayed
4,xk-9927,3349,shipped


In [151]:
# Let's join the table together!
tx_shipment_df = pd.merge(tx_df, shipment_df, on='transaction_id')
tx_shipment_df

,transaction_id,quarter,customer_name,transit_code,order_total_amt,process_flag,shipment_id,shipment_status
0,3345,Q1,Robert Robertson,X4,100,1,xk-9923,in_process
1,3346,Q1,David Davison,Y4,230,1,xk-9924,shipped
2,3347,Q1,Jack Froot,X5,332,1,xk-9925,shipped
3,3348,Q1,Dolby Sorand,Y5,50,1,xk-9926,delayed
4,3349,Q1,Robert Robertson,X6,65,1,xk-9927,shipped
5,3350,Q1,David Davison,Y6,80,1,xk-9928,shipped
6,3351,Q1,Jack Froot,X7,125,1,xk-9929,in_process
7,3352,Q1,Dolby Sorand,Y7,225,2,xk-9930,shipped
8,3353,Q1,Robert Robertson,X8,347,2,xk-9931,shipped
9,3354,Q1,David Davison,Y8,567,0,xk-9932,shipped


In [152]:
# do we have all the data?
print(len(tx_df))
print(len(shipment_df))
print(len(tx_shipment_df))

55
41
41


In [157]:
# Why does this matter? Let's sum up the order totals.
print('Transaction Total:', tx_df.order_total_amt.sum())
print('Transaction Shipment Total:', tx_shipment_df.order_total_amt.sum())

Transaction Total: 12974
Transaction Shipment Total: 9860


In [160]:
# We need to do an outer join
tx_shipment_outer_df = pd.merge(tx_df, shipment_df, on='transaction_id', how='left')
print(len(tx_shipment_outer_df))
print('Transaction Shipment Outer Total:', tx_shipment_outer_df.order_total_amt.sum())

55
Transaction Shipment Outer Total: 12974


In [161]:
tx_shipment_outer_df

,transaction_id,quarter,customer_name,transit_code,order_total_amt,process_flag,shipment_id,shipment_status
0,3345,Q1,Robert Robertson,X4,100,1,xk-9923,in_process
1,3346,Q1,David Davison,Y4,230,1,xk-9924,shipped
2,3347,Q1,Jack Froot,X5,332,1,xk-9925,shipped
3,3348,Q1,Dolby Sorand,Y5,50,1,xk-9926,delayed
4,3349,Q1,Robert Robertson,X6,65,1,xk-9927,shipped
5,3350,Q1,David Davison,Y6,80,1,xk-9928,shipped
6,3351,Q1,Jack Froot,X7,125,1,xk-9929,in_process
7,3352,Q1,Dolby Sorand,Y7,225,2,xk-9930,shipped
8,3353,Q1,Robert Robertson,X8,347,2,xk-9931,shipped
9,3354,Q1,David Davison,Y8,567,0,xk-9932,shipped


## Excel Lyfe

### Vlookup

In [165]:
vlookup_tx_df = pd.read_excel('vlookup-example_raw.xlsx', sheet_name='transaction')
vlookup_prod_df = pd.read_excel('vlookup-example_raw.xlsx', sheet_name='product_lookup')


In [166]:
vlookup_tx_df

,transaction_id,product_id
0,11234,1
1,11235,4
2,11236,4
3,11237,2
4,11238,3
5,11239,1
6,11240,5
7,11241,1
8,11242,3
9,11243,2


In [167]:
vlookup_prod_df

,product_id,product_name
0,1,Turnips
1,2,Mushrooms
2,3,Bananas
3,4,Cherries
4,5,Peaches


In [168]:
vlookup_df = pd.merge(vlookup_tx_df, vlookup_prod_df, how='inner', on='product_id')

In [169]:
vlookup_df

,transaction_id,product_id,product_name
0,11234,1,Turnips
1,11239,1,Turnips
2,11241,1,Turnips
3,11246,1,Turnips
4,11247,1,Turnips
5,11235,4,Cherries
6,11236,4,Cherries
7,11249,4,Cherries
8,11237,2,Mushrooms
9,11243,2,Mushrooms


### Pivot Tables

In [172]:
tx_df.head()

,transaction_id,quarter,customer_name,transit_code,order_total_amt,process_flag
0,3345,Q1,Robert Robertson,X4,100,1
1,3346,Q1,David Davison,Y4,230,1
2,3347,Q1,Jack Froot,X5,332,1
3,3348,Q1,Dolby Sorand,Y5,50,1
4,3349,Q1,Robert Robertson,X6,65,1


In [178]:
pd.pivot_table(tx_df, values='order_total_amt', index=['quarter'], columns=['customer_name'], aggfunc=np.sum)

customer_name,David Davison,Dolby Sorand,Jack Froot,Robert Robertson
quarter,,,,
Q1,1187,1093,1248,1024
Q2,1674,1361,1125,612
Q3,647,275,791,412
Q4,310,593,457,165


In [179]:
pd.pivot_table(tx_df, values='order_total_amt', index=['quarter', 'customer_name'], aggfunc=np.sum)

order_total_amt
quarter customer_name                    
Q1      David Davison                1187
        Dolby Sorand                 1093
        Jack Froot                   1248
        Robert Robertson             1024
Q2      David Davison                1674
        Dolby Sorand                 1361
        Jack Froot                   1125
        Robert Robertson              612
Q3      David Davison                 647
        Dolby Sorand                  275
        Jack Froot                    791
        Robert Robertson              412
Q4      David Davison                 310
        Dolby Sorand                  593
        Jack Froot                    457
        Robert Robertson              165